In [1]:
from generator import *

2023-05-30 10:31:20.583216: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-30 10:31:20.775476: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-30 10:31:20.777221: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-30 10:31:21.575812: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
[nltk_data] Downloading package wordnet to /home/jj/nltk_data..

In [16]:
# command = "!generate ass"
# content_generator(command)

In [2]:
from keras.layers import Input, Dense, Activation, TimeDistributed, Softmax, TextVectorization, Reshape, RepeatVector, Conv1D, Bidirectional, AveragePooling1D, UpSampling1D, Embedding, Concatenate, GlobalAveragePooling1D, LSTM, Multiply
from keras.models import Model
import tensorflow as tf
import keras
import numpy as np
import random

import pandas as pd
import os
import re
from tqdm import tqdm

from keras.layers import Input, TextVectorization
from keras.models import Model

In [40]:
DATASET = 'crawler_data.csv'
df = pd.read_csv(DATASET)

ds = tf.data.Dataset.from_tensor_slices(df)

vocab_size = 5000
vectorize_layer = TextVectorization(max_tokens=vocab_size, output_sequence_length=10)
vectorize_layer.adapt(ds)

def separar_ultimo_token(x):
    x_ = vectorize_layer(x)
    x_ = x_[:,:-1]
    y_ = x_[:,-1:]
    return x_, y_

def predict_word(seq_len, latent_dim, vocab_size):
    input_layer = Input(shape=(seq_len-1,))
    x = input_layer
    x = Embedding(vocab_size, latent_dim, name='embedding', mask_zero=True)(x)
    x = LSTM(latent_dim, kernel_initializer='glorot_uniform')(x)
    latent_rep = x
    x = Dense(vocab_size)(x)
    x = Softmax()(x)
    return Model(input_layer, x), Model(input_layer, latent_rep)

predictor, latent = predict_word(10, 15, vocab_size)
predictor.summary()
#opt = keras.optimizers.SGD(learning_rate=1, momentum=0.9)
opt = keras.optimizers.Nadam(learning_rate=0.1)
loss_fn = keras.losses.SparseCategoricalCrossentropy(
    ignore_class=1,
    name="sparse_categorical_crossentropy",
)

predictor.compile(loss=loss_fn, optimizer=opt, metrics=["accuracy"])

2023-05-30 10:58:28.465366: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [259,3]
	 [[{{node Placeholder/_0}}]]


Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 9)]               0         
                                                                 
 embedding (Embedding)       (None, 9, 15)             75000     
                                                                 
 lstm_5 (LSTM)               (None, 15)                1860      
                                                                 
 dense_5 (Dense)             (None, 5000)              80000     
                                                                 
 softmax_5 (Softmax)         (None, 5000)              0         
                                                                 
Total params: 156,860
Trainable params: 156,860
Non-trainable params: 0
_________________________________________________________________


In [43]:
# series = pd.DataFrame(df["body"])

# ds = tf.data.Dataset.from_tensor_slices(series)

# # series = pd.Series(series.all())
# series

# series = series.dropna()
# series = series.drop_duplicates()
tf.config.run_functions_eagerly(True)

history = predictor.fit(ds.map(separar_ultimo_token), epochs=20, verbose=1)


Epoch 1/20
  1/259 [..............................] - ETA: 32s - loss: 8.4003 - accuracy: 0.3333

/home/jj/.local/lib/python3.8/site-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
2023-05-30 10:59:55.318426: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [259,3]
	 [[{{node Placeholder/_0}}]]


259/259 [==============================] - 13s 50ms/step - loss: 2.4579 - accuracy: 0.6564
Epoch 2/20
259/259 [==============================] - 13s 50ms/step - loss: 1.6273 - accuracy: 0.6937
Epoch 3/20
259/259 [==============================] - 13s 50ms/step - loss: 1.3197 - accuracy: 0.7207
Epoch 4/20
259/259 [==============================] - 13s 49ms/step - loss: 1.3377 - accuracy: 0.7104
Epoch 5/20
259/259 [==============================] - 13s 49ms/step - loss: 1.2284 - accuracy: 0.7272
Epoch 6/20
259/259 [==============================] - 13s 50ms/step - loss: 1.0507 - accuracy: 0.7516
Epoch 7/20
259/259 [==============================] - 13s 49ms/step - loss: 1.1285 - accuracy: 0.7284
Epoch 8/20
259/259 [==============================] - 13s 50ms/step - loss: 1.1451 - accuracy: 0.7503
Epoch 9/20
259/259 [==============================] - 13s 50ms/step - loss: 1.1209 - accuracy: 0.7619
Epoch 10/20
259/259 [==============================] - 13s 49ms/step - loss: 0.8634 - accurac

In [53]:
pred = predictor.predict(vectorize_layer(["eu gosto de"])[:,:-1])
print(pred)
idx = tf.argmax(pred, axis=1)[0]
print(idx)
word = vectorize_layer.get_vocabulary()[idx]
print(word)

1/1 [==============================] - 0s 22ms/step
[[3.9052212e-04 1.7625330e-09 2.7955217e-02 ... 1.7424109e-09
  1.7431023e-09 1.7308673e-09]]
tf.Tensor(762, shape=(), dtype=int64)
jump


In [73]:

def beam_search_predizer(entrada, numero_de_predicoes, modelo, vectorize_layer, beam_size):
    frase = entrada
    contexto = frase  # Contexto deslizante
    for n in range(numero_de_predicoes):
        pred = modelo.predict(vectorize_layer([contexto])[:, :-1])

        # Beam search
        candidates = []
        for _ in range(beam_size):
            # Select top candidate from the predictions
            idx = np.argmax(pred, axis=1)[0]
            word = vectorize_layer.get_vocabulary()[idx]

            # Check if the word is already in the sentence
            if word not in frase.split():
                candidates.append((word, pred[0, idx]))

            # Set the probability of the selected word to 0
            pred[0, idx] = 0

        # Sort candidates based on probability in descending order
        candidates.sort(key=lambda x: x[1], reverse=True)

        # Select the top candidate as the next word
        word = candidates[0][0]

        frase = frase + " " + word
        contexto = contexto + " " + word
        contexto = ' '.join(contexto.split()[1:])
        print(word)

    return frase


beam_search_predizer("brasil ", 15, predictor, vectorize_layer, beam_size=20)

1/1 [==============================] - 0s 28ms/step
home
1/1 [==============================] - 0s 25ms/step
content
1/1 [==============================] - 0s 24ms/step
main
1/1 [==============================] - 0s 29ms/step
you
1/1 [==============================] - 0s 33ms/step
меню
1/1 [==============================] - 0s 23ms/step

1/1 [==============================] - 0s 27ms/step

1/1 [==============================] - 0s 31ms/step

1/1 [==============================] - 0s 23ms/step

1/1 [==============================] - 0s 29ms/step

1/1 [==============================] - 0s 23ms/step

1/1 [==============================] - 0s 23ms/step

1/1 [==============================] - 0s 28ms/step

1/1 [==============================] - 0s 31ms/step

1/1 [==============================] - 0s 27ms/step



'brasil  home content main you меню          '

In [77]:
command = "!generate: brasil"

match = re.match(r"!generate:(.+)", command)
print(match.group(1))

 brasil


In [1]:
from generator import content_generator

2023-05-30 11:47:53.505034: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-30 11:47:53.530616: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-30 11:47:53.530980: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-30 11:47:54.130815: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2023-05-30 11:47:54.852500: E tensorflow/compiler/xla/stream_ex

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 9)]               0         
                                                                 
 embedding (Embedding)       (None, 9, 15)             75000     
                                                                 
 lstm (LSTM)                 (None, 15)                1860      
                                                                 
 dense (Dense)               (None, 5000)              80000     
                                                                 
 softmax (Softmax)           (None, 5000)              0         
                                                                 
Total params: 156,860
Trainable params: 156,860
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20


/home/jj/.local/lib/python3.8/site-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
2023-05-30 11:47:55.828806: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [259,3]
	 [[{{node Placeholder/_0}}]]


 53/259 [=====>........................] - ETA: 10s - loss: 2.3645 - accuracy: 0.6352

In [ ]:
print(content_generator("!generate: brasil"))